# Database Example  

In [2]:
# Import libraries
import aopy
from aopy.data import db
from ipywidgets import interactive, widgets
from datetime import date, datetime, timedelta
import re
from tqdm.notebook import tqdm
import numpy as np

In [3]:
# set up constants
preproc_dir = '/data/preprocessed'

## Finding a single recording

In [3]:
subject = 'beignet'
task = 'manual control'
project = 'baseline flash'
day = date(2022, 6, 11) # June 11, 2022

sessions = db.lookup_sessions(subject=subject, task_name=task, date=day, project=project)
e = sessions[-1]
print(e)
print(e.project)

2022-06-11 09:55:10.450000: beignet on manual control task, id=5624
baseline flash


In [4]:
# Filter on recordings with neural data
filter_fn = db.filter_has_neural_data('ecog')
sessions = db.lookup_sessions(subject='affi', task_desc='resting data', experimenter='leo', filter_fn=filter_fn)
print(f"{len(sessions)} sessions found with ecog data")

412 sessions found with ecog data


In [5]:
# Filter specific parameters
filter_fn = lambda x: x.get_task_param('reward_time') == 0.25
sessions = db.lookup_sessions(subject='affi', filter_fn=filter_fn)
print(f"{len(sessions)} sessions found with 250ms rewards")

197 sessions found with 250ms rewards


In [6]:
# Or use a shortcut
sessions = db.lookup_mc_sessions(subject=subject, date=('2023-06-05', '2023-06-10'))
subjects, ids, dates = db.list_entry_details(sessions)
print(subjects, ids, dates)

('beignet', 'beignet', 'beignet', 'beignet') (9609, 9631, 9638, 9667) (datetime.date(2023, 6, 5), datetime.date(2023, 6, 6), datetime.date(2023, 6, 8), datetime.date(2023, 6, 9))


## Finding a range of dates

In [7]:
start = widgets.DatePicker(
    description='Start date',
    disabled=False
)
end = widgets.DatePicker(
    description='End date',
    disabled=False
)
display(start)
display(end)

DatePicker(value=None, description='Start date', step=1)

DatePicker(value=None, description='End date', step=1)

In [9]:
# Might want to do some additional filtering by date
subject = 'beignet'
task = 'bmi control'
project = 'linear map'
entries = db.lookup_sessions(subject=subject, task_name=task, date=(start.value, end.value), project=project)
print(f"{len(entries)} matching recordings")

101 matching recordings


## Select from a dropdown menu

In [10]:
entry_drop = widgets.Dropdown(options=entries)
display(entry_drop)

Dropdown(options=(2022-06-06 13:44:42.434000: beignet on bmi control task, id=5512, 2022-06-06 13:45:31.687000…

In [11]:
e = entry_drop.value
print(e.date)
print(e.datetime)

2022-06-06
2022-06-06 13:44:42.434000


## Select anything

In [12]:
start_date = widgets.DatePicker(value = datetime.today() - timedelta(days=30))
end_date = widgets.DatePicker(value = datetime.today())

# Filter by task and subject
entries = db.lookup_sessions(date=(start_date.value, end_date.value))
subjects = np.unique([e.subject for e in entries])
tasks = np.unique([e.task_name for e in entries])

subj_drop = widgets.Dropdown(options=subjects)
task_drop = widgets.Dropdown(options=tasks) 
entry_select = widgets.SelectMultiple(
    options=[],
    description='entries',
    disabled=False
)
entry_select.layout.width = '600px'

def update_entries(*args):
    entries = db.lookup_sessions(
        subject=subj_drop.value, 
        task_name=task_drop.value,
        date=(start_date.value,end_date.value)
    )
    entry_select.options = ["{} - {}: {}".format(e.id, e.task_desc, e.date) for i, e in enumerate(entries)]
    
start_date.observe(update_entries)
end_date.observe(update_entries)
subj_drop.observe(update_entries)
task_drop.observe(update_entries)

def select_entry(subject, task, start_date, end_date, entries):
    ids = [int(re.search(r'\d+', e).group()) for e in entries]
    return db.lookup_sessions(id=ids)


selection = interactive(select_entry,
                     subject=subj_drop,
                     task=task_drop,
                     start_date=start_date,
                     end_date=end_date,
                     entries=entry_select)

display(selection)

interactive(children=(Dropdown(description='subject', options=('affi', 'churro', 'test'), value='affi'), Dropd…

In [13]:
entries = selection.result
print(f"{len(entries)} selected")

1 selected


## Loading the data

In [14]:
for e in tqdm(entries):
    
    # Load the data
    exp_data, exp_metadata = aopy.data.load_preproc_exp_data(preproc_dir, e.subject, e.id, e.date)

    print(f"entry {e.id} has {len(exp_data['clock'])} clock cycles")

  0%|          | 0/1 [00:00<?, ?it/s]

entry 17676 has 4256 clock cycles


## Setting filter function to a defined function

In [4]:
db.BMI3D_DBNAME = 'booth' 

In [5]:
def filter_for_desired_sessions(e):
    
    value = (
        e.has_feature('optitrack') and 
        'flash' not in e.task_desc and 
        'centerout_2D' in e.sequence_name and 
        e.subject != 'test'
        )
    
    try:
        exp_data, exp_metadata = aopy.data.load_preproc_exp_data(preproc_dir, e.subject, e.id, e.date)
        t = exp_data['events']['timestamp']
        
        #hand_offset exp_meta
        
    except:
        return False
    
    return value

In [6]:
#constants
preproc_dir = '/data/preprocessed'

affi_dates = ('2021-12-02','2021-12-27')

affi_sessions = db.lookup_sessions(task_name='manual control',
                              subject = 'affi',
                              date = affi_dates,
                              filter_fn = filter_for_desired_sessions) #func that finds all manual control sessions and returns as an array

In [7]:
display(affi_sessions)

[2021-12-02 14:23:35.041000: affi on manual control task, id=3407,
 2021-12-02 14:24:38.642000: affi on manual control task, id=3408,
 2021-12-16 11:59:21.893000: affi on manual control task, id=3545,
 2021-12-16 12:01:12.051000: affi on manual control task, id=3546,
 2021-12-17 12:40:20.725000: affi on manual control task, id=3567,
 2021-12-17 12:57:34.536000: affi on manual control task, id=3568,
 2021-12-17 13:02:13.886000: affi on manual control task, id=3569,
 2021-12-20 13:37:26.169000: affi on manual control task, id=3593,
 2021-12-20 13:40:50.615000: affi on manual control task, id=3594,
 2021-12-20 13:43:03.629000: affi on manual control task, id=3595,
 2021-12-21 13:04:31.829000: affi on manual control task, id=3611,
 2021-12-21 13:06:16.866000: affi on manual control task, id=3612,
 2021-12-23 13:19:47.602000: affi on manual control task, id=3613]

## Querying a different rig

In [15]:
db.BMI3D_DBNAME = 'tablet' # booth, tablet, human, test
subject = 'churro'

sessions = db.lookup_sessions(subject=subject)
e = sessions[-1]
print(e)
print(e.project)

2024-07-08 14:55:05.579804: churro on tracking task task, id=703
bumpers
